# E-ML - Phishing Email Detection with Machine Learning Methods
### By Colin Harrison, Aidan Huerta, and Drew Levy
--- 

In [3]:
print("Importing necessary libraries...\n")
from nltk import word_tokenize
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

from sklearn import linear_model
from sklearn import metrics

from sklearn.metrics import accuracy_score, roc_auc_score

import datasets
import re

import nltk
import sklearn
import numpy as np
from nltk import FreqDist, word_tokenize
from nltk import pos_tag
from typing import List, Tuple, Optional

import warnings
warnings.simplefilter('ignore')

from typing import Any, Dict, Iterable, List, Optional, Set, Tuple

# Might be necessary
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('universal_tagset')
print("Done!")

Importing necessary libraries...

Done!


[nltk_data] Downloading package punkt_tab to /home/drewl/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/drewl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/drewl/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [7]:
data_df = pd.read_csv('Phishing_Email.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 70)
pd.set_option('display.expand_frame_rep', False)
print("Phishing Data shape:", data_df.shape)
print()
print("Below is a small sample of what the data we will be using looks like:")
print(data_df.sample(n=5))

Phishing Data shape: (18650, 3)

Below is a small sample of what the data we will be using looks like:
       Unnamed: 0                                                             Email Text      Email Type
13796       13797  On Tue, 2002-10-08 at 10:36, Matthias Saou wrote:\n> Hi there,\n> ...      Safe Email
15618       15619  dowlnoadable 70 + xxx vldeos with pornstars - x 41 welcome to the ...  Phishing Email
7436         7437  paypal account verification notification of limited account access...  Phishing Email
967           967  re : new carthage phys product for eol for the eol location cartha...      Safe Email
7622         7623  accomplishments for year 2001 , as of 05 / 10 revenue generating :...      Safe Email
